In [7]:
import numpy as np
import scipy.cluster.hierarchy as hcluster
import matplotlib.pyplot as plt
import os
import cv2
import sys
from scipy.spatial import distance
from centroidtracker import CentroidTracker
from trackableobject import TrackableObject
%matplotlib
#Write down conf, nms thresholds,inp width/height
confThreshold = 0.75
nmsThreshold = 0.40
inpWidth = 576
inpHeight = 768
X = 580
Y = 80
Width = 730
Height = 400
W = None
H = None

#Load names of classes and turn that into a list
classesFile = "coco.names"
classes = None

with open(classesFile,'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

#Model configuration
modelConf = 'yolov3.cfg'
modelWeights = 'yolov3.weights'


def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

#Set up the net
net = cv2.dnn.readNetFromDarknet(modelConf, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

ct = CentroidTracker(maxDisappeared=5, maxDistance=50)
trackableObjects = {}
totalDown = 0
totalUp = 0
enteredObject =[]
exitedObject = []
colour_code = {}
dist_threshold = 1
fn = '000000' 
for count in range(229,len(os.listdir('sequence'))):
    a = ('sequence/'+fn[len(str(count)):]+str(count)+'.jpg')
    b = ('res_out_clustering/'+fn[len(str(count)):]+str(count)+'.jpg')
    frame = cv2.imread(a)#cv2.imread('images/000051.jpg')#cv2.VideoCapture(0)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    rects = []
    blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop = False)
    net.setInput(blob)
    outs = net.forward (getOutputsNames(net))
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    classIDs = []
    confidences = []
    boxes = []
    status = "People in Group"
    for out in outs:
        for detection in out:            
            scores = detection [5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confThreshold:
                centerX = int(detection[0] * frameWidth)
                centerY = int(detection[1] * frameHeight)
                width = int(detection[2]* frameWidth)
                height = int(detection[3]*frameHeight )
                left = int(centerX - width/2)
                top = int(centerY - height/2)
                classIDs.append(classID)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
    indices = cv2.dnn.NMSBoxes (boxes,confidences, confThreshold, nmsThreshold )
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        startX = box[0]
        startY = box[1]
        endX = box[2] + box[0]
        endY = box[3] + box[1]
        classId = classIDs[i]
        conf = confidences[i]
        label = '%.2f' % conf
        if classes:
            assert (classId < len(classes))
            label = '%s:%s' % (classes[classId], label)
        if label.startswith('person'):
            rects.append((startX, startY, endX, endY))
    objects = ct.update(rects)
    print(a)
    centroidList = list(objects.values())
    clusters = hcluster.fclusterdata(centroidList,t=0.1) #need to adjust threshold
    clusters = np.asarray(clusters)

    def draw_dot_single(indexList):
        for i in indexList:
            cv2.circle(frame, (int(centroidList[i][0]),int(centroidList[i][1])), 3, (0, 0, 255), 3)
            
    def draw_dot_single_1(centroidList):
        for i in centroidList:
            cv2.circle(frame, (int(i[0]),int(i[1])), 3, (0, 0, 255), 3)
            
    def draw_line(cList,indexList):
        groupCentroid = np.mean(cList, axis=0)   
#         cv2.circle(frame, (int(groupCentroid[0]),int(groupCentroid[1])), 3, (0, 0, 255), 3)
        for i in indexList:
            cv2.circle(frame, (int(centroidList[i][0]),int(centroidList[i][1])), 3, (0, 0, 255), 3)
            for j in indexList:
                cv2.line(frame, (int(centroidList[i][0]),int(centroidList[i][1])),
                     (int(centroidList[j][0]),int(centroidList[j][1])), (0, 255, 0), 2)
            
        
    groups = dict(zip(*np.unique(clusters, return_counts=True)))
    single=0
    n=1
    count = set()
    info = [("Status", status)]
    if len(groups) ==1:
        # print("No of single pedestrians :",len(clusters))
        info.append(("count for group" , single))
        info.append(("No of single pedestrians" , len(clusters)))
        draw_dot_single_1(centroidList)
    else:

        for k,v in groups.items():
            if v==1:
                single+=1
                indexList = np.where(clusters == k)[0]
                draw_dot_single(indexList)
            else:
                info.append((f"count for group {n}" , v))
                n+=1
                count.add(v)

            indexList = np.where(clusters == k)[0]
            draw_line([centroidList[i] for i in indexList], indexList)
        
        info.append(("No of single pedestrians" , single))
        
    rect_x,rect_y,rect_w,rect_h = 0,0,250,80
    cv2.rectangle(frame, (rect_x, rect_x), (rect_x + rect_w, rect_y + rect_h), (0,0,0), -1)
#     # loop over the info tuples and draw them on our frame
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(frame, text, (10, ((i * 20) + 15)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imwrite(b, frame)
#     break
cv2.destroyAllWindows()

Using matplotlib backend: Qt5Agg
sequence/000229.jpg
sequence/000230.jpg
sequence/000231.jpg
sequence/000232.jpg
sequence/000233.jpg
sequence/000234.jpg
sequence/000235.jpg
sequence/000236.jpg
sequence/000237.jpg
sequence/000238.jpg
sequence/000239.jpg
sequence/000240.jpg
sequence/000241.jpg
sequence/000242.jpg
sequence/000243.jpg
sequence/000244.jpg
sequence/000245.jpg
sequence/000246.jpg
sequence/000247.jpg
sequence/000248.jpg
sequence/000249.jpg
sequence/000250.jpg
sequence/000251.jpg
sequence/000252.jpg
sequence/000253.jpg
sequence/000254.jpg
sequence/000255.jpg
sequence/000256.jpg
sequence/000257.jpg
sequence/000258.jpg
sequence/000259.jpg
sequence/000260.jpg
sequence/000261.jpg
sequence/000262.jpg
sequence/000263.jpg
sequence/000264.jpg
sequence/000265.jpg
sequence/000266.jpg
sequence/000267.jpg
sequence/000268.jpg
sequence/000269.jpg
sequence/000270.jpg
sequence/000271.jpg
sequence/000272.jpg
sequence/000273.jpg
sequence/000274.jpg
sequence/000275.jpg
sequence/000276.jpg
sequenc

sequence/000637.jpg
sequence/000638.jpg
sequence/000639.jpg
sequence/000640.jpg
sequence/000641.jpg
sequence/000642.jpg
sequence/000643.jpg
sequence/000644.jpg
sequence/000645.jpg
sequence/000646.jpg
sequence/000647.jpg
sequence/000648.jpg
sequence/000649.jpg
sequence/000650.jpg
sequence/000651.jpg
sequence/000652.jpg
sequence/000653.jpg
sequence/000654.jpg
sequence/000655.jpg
sequence/000656.jpg
sequence/000657.jpg
sequence/000658.jpg
sequence/000659.jpg
sequence/000660.jpg
sequence/000661.jpg
sequence/000662.jpg
sequence/000663.jpg
sequence/000664.jpg
sequence/000665.jpg
sequence/000666.jpg
sequence/000667.jpg
sequence/000668.jpg
sequence/000669.jpg
sequence/000670.jpg
sequence/000671.jpg
sequence/000672.jpg
sequence/000673.jpg
sequence/000674.jpg
sequence/000675.jpg
sequence/000676.jpg
sequence/000677.jpg
sequence/000678.jpg
sequence/000679.jpg
sequence/000680.jpg
sequence/000681.jpg
sequence/000682.jpg
sequence/000683.jpg
sequence/000684.jpg
sequence/000685.jpg
sequence/000686.jpg


In [1]:
import os
import cv2
img_array = []
fn = '000000' 
for count in range(1,len(os.listdir('res_out'))):
    a = ('res_out_clustering/'+fn[len(str(count)):]+str(count)+'.jpg')
    img = cv2.imread(a)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('Task3_clustering.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [165]:
a = [[0, 3, 1], [4, 7]]


5
